In [1]:
def int2rgb(n):
    b = n % 256
    g = int( ((n-b)/256) % 256 )      # always an integer
    r = int( ((n-b)/256**2) - g/256 ) # ditto
    return (str(r/255),str(g/255),str(b/255))


In [3]:
#load streams from Adrian and show them in openspace
import requests
import json
from matplotlib.colors import to_rgb
import urllib.request

with urllib.request.urlopen("http://adrian.pw/visualizations/mwstreams/data.json") as url:
    streams = json.loads(url.read().decode())

#f = open('mwstreams.json')
#streams = json.loads(f)
#f.close()


In [13]:
keys = []
asset = "local transforms = asset.require('scene/solarsystem/sun/transforms')"
for stream_key in streams:
    stream = streams[stream_key]
    stream_name = stream_key.replace('-', ' ')
    stream_name = stream_name.replace(' ', '')
    print(stream_key);
    color = stream["color"]
    rgb_color = int2rgb(int(color,16))
    
    speck = ""
    scale = 1
    for star in stream["data"]:
        speck += str(star[0]*scale) + "\t" + str(star[1]*scale) + "\t" + str(star[2]*scale) + "\t0" +  "\n"

    speck_file = open(stream_name + ".speck", "w")
    speck_file.write(speck)
    speck_file.close()

    #get values for presentation
    stream_opacity = 1.0
    if 'opacity' in stream:
        stream_opacity = stream["opacity"]
    stream_size = 0.35
    if stream_name == 'Disk':
        stream_size = 0.1
    if 'size' in stream:
        stream_size = stream['size']
    #arbitrary starting value of 405 based on other assets
    stream_scale_factor = 430 + 20 * stream_size

    #create asset lua
    asset += "local " + stream_name + " = {\n\
    Identifier = 'MilkyWayStreams" + stream_name + "',\n\
    Parent = 'Root',\n\
    Transform = {\n\
        Translation = {\n\
            Type = 'StaticTranslation',\n\
            -- The center of the Milky Way is approximately 8 kiloparsec from the Sun.\n\
            -- The x-axis of galactic coordinates points from the sun towards the center\n\
            -- of the galaxy.\n\
            Position = { 8 * 3.086E19, 0, 0 }\n\
        },\n\
    },\n\
    Renderable = {\n\
        Type = 'RenderableBillboardsCloud',\n\
        Enabled = true,\n\
        Color = {" + rgb_color[0] + "," + rgb_color[1] + ", " + rgb_color[2] + "},\n\
        Opacity = " + str(stream_opacity) + ",\n\
        File = asset.localResource('" + stream_name + ".speck'),\n\
        Texture = asset.localResource('point3A.png'),\n\
        PolygonSides = 6,\n\
        Unit = 'Kpc',\n\
        ScaleFactor = " + str(stream_scale_factor) + ",\n\
        BillboardMinMaxSize = { 0.0, 300.0 },\n\
        EnablePixelSizeControl = false\n\
    },\
    GUI = {\n\
        Name = '" + stream_name + "',\n\
        Path = '/Milky Way/Streams'\n\
    }\n\
}\n\n"

    keys.append(stream_name)
#end loop

asset += "local toggleStreamsAction = {\n\
  Identifier = 'toggleStreams',\n\
  Name = 'Toggle Milky Way Stream Visibility',\n\
  Command = [[\n\
    local currentFade = openspace.getPropertyValue('Scene.MilkyWayStreamsDisk.Renderable.Fade')\n\
    local newFade = 0\n\
    local easeFunction = ''\n\
    if currentFade < 1 then\n\
      newFade = 1\n\
      easeFunction = 'CubicEaseOut'\n\
    end\n\
    openspace.setPropertyValue('Scene.MilkyWayStreams*.Renderable.Fade', newFade, 3, easeFunction)\n\
  ]],\n\
  GuiPath = '/Milky Way/Streams',\n\
  IsLocal = true\n\
}\n\n"    

asset += "asset.onInitialize(function()\n"
asset += "\topenspace.action.registerAction(toggleStreamsAction)\n"
for name in keys:
    asset += f"\topenspace.addSceneGraphNode({name})\n"
asset += "end)\n\n"

asset += "asset.onDeinitialize(function()\n"
asset += "\topenspace.action.removeAction(toggleStreamsAction)\n"
for name in keys:
    asset += f"\topenspace.removeSceneGraphNode({name})\n"
asset += "end)\n\n"


asset_file = open("mwstreams.asset", "w")
asset_file.write(asset)
asset_file.close()

Disk
MagellanicClouds
Sagittarius
GD-1
Orphan
Pal 5
Ophiuchus
ATLAS
Aliqa Uma
Chenab
Elqui
Indus
Jhelum
Phoenix
Tucana III
Turranburra


In [4]:
#get full path for script command
import os
asset_full_path = os.path.realpath(asset_file.name)



#connect to openspace 
from websocket import create_connection
import json
url = "localhost:4682"
ws_endpoint = f"ws://localhost:4682/websocket"
ws = create_connection(ws_endpoint)
#command to add asset
message = json.dumps({"topic": 4,
                  "type": "luascript",
                  "payload": {"function": "openspace.asset.add",
                              "arguments": [asset_full_path],
                              "return": False}})
ws.send(message)
#refresh menu
message = json.dumps({"topic": 4,
                  "type": "luascript",
                  "payload": {"function": "openspace.setPropertyValueSingle",
                              "arguments": ['Modules.CefWebGui.Reload', None],
                              "return": False}})
ws.send(message)
#ws.close()

168

In [5]:
#remove asset if you want to change values and try again
message = json.dumps({"topic": 4,
                  "type": "luascript",
                  "payload": {"function": "openspace.asset.remove",
                              "arguments": [asset_full_path],
                              "return": False}})
ws.send(message)
ws.close()